В блокноте рассматриваются два способа записи рекуррентного уравнения ДП (уравнения Беллмана) и, соответственно, два способа организации вычислений: обратная прогонка и прямая прогонка.

Большинство других методических материалов по курсу:
- [Блокнот Dynamic Programming](https://bitbucket.org/M_H/dmt-examples/src/master/Dynamic%20Programming.ipynb)
- [Блокнот Dynamic Programming (continuous)](https://bitbucket.org/M_H/dmt-examples/src/master/Dynamic%20Programming%20(continuous).ipynb)
- [Методичка по ДП на Octave](https://avponomarev.bitbucket.io/DP_Octave.pdf)

используют обратную прогонку, однако для ряда задач (и ряда людей) уравнения прямой прогонки оказываются более логичными.

# Задача

В качестве задачи, для которой мы построим два вида рекуррентных уравнений, рассмотрим задачу о найме (из Х. Таха Введение в исследование операций. 6-е издание):

Строительный подрядчик оценивает минимальные потребности в рабочей силе на каждую из последующих пяти недель следующим образом: 5, 6, 8, 4 и 6 рабочих соответственно. Содержание избытка рабочей силы обходится подрядчику в 300 долларов за одного рабочего в неделю, а наем рабочей силы на протяжении одной недели обходится в 400 долларов плюс 200 долларов за одного рабочего в неделю.


In [1]:
# Требуемое количество рабочих для каждого этапа (каждой недели)
workforce_demand = [5, 7, 8, 4, 6]

# Обратная прогонка

В любом случае, формализуя задачу для решения с помощью динамического программирования, мы представляем ее как многоэтапный процесс принятия решений, в ходе которого на каждом из *этапов* мы можем оказываться в некотором *состоянии*, причем переход из одного состояния в другое связан с определенным *выигрышем*. (Примечание: может показаться, что эта задача очень близка к поиску оптимального пути в графе состояний задачи - так оно и есть. Более того, алгоритм Форда-Беллмана представляет собой применение принципа динамического программирования к поиску кратчайшего пути.) И, опять-таки, в любом случае, мы строим оптимальную траекторию с использованием оптимальных же траекторий меньшей "длины" (за меньшее число этапов). 

Уравнения обратной прогонки записываются для выигрыша, начиная с $i$-того этапа, который рекуррентно выражается через выигрыш, начиная с $i+1$-го этапа. Соответственно, в самом уравнении рассматривается переход от состояния, в котором система оказывается к началу $i$-того этапа, к состоянию, в котором система окажется к началу следующего, $i+1$-го этапа под действием того или иного управления. 

<img src="images/dp_backward.png" alt="Обратная прогонка" style="width: 600px;"/>

На рисунке кружками обозначены состояния. Считается, что независимо от этапа, на каждом состоянии возможно предпринять одно из трех возможных управлений $u^{(1)}$, $u^{(2)}$ и $u^{(3)}$ (эффект от применения управления показан с помощью стрелок). Выигрыш от осуществления определенного управления при условии нахождения системы в том или ином состоянии обозначен как $w_i(s, u)$, где $s$ и $u$ - состояние и управление соответственно. $W_i(s)$ - (суммарный) условный оптимальный выигрыш для всех этапов, начиная с $i$-того, при условии, что к началу $i$-того этапа система находится в состоянии $s$. Обратите внимание, что состояние, на самом деле, находится между рассматриваемыми этапами (в ходе которых применяется то или иное управление), это порождает несколько вариантов при записи соотношений - считать это состояние конечным состоянием этапа или начальным. Уравнения обратной прогонки выглядят более логичными, если считать, что это состояние *к началу* соответствующего этапа. 

Поскольку ДП применяется к задачам, которые удовлетворяют принципу *оптимальности подзадач*, оптимальный путь по пространству состояний задачи начиная от состояния $s$ на этапе $i$ включает в себя оптимальный путь по пространству состояний задачи от некоторого состояния $s'$, достижимого из $s$ применением того или иного управления на этапе $i$. Соответственно, мы можем попытаться выразить $W_i(s)$ через $W_{i+1}(s')$ и стоимости управлений, которые приводят из $s$ в $s'$. Так, на приведенной картинке, оценка оптимального управления на всех шагах, начиная с $i$-того, при условии, что система к $i$-тому шагу находится в состоянии 2 (то есть, $W_i(2)$), опирается на оценку оптимального управления для последующих шагов для состояний, в которые возможно попасть из состояния 2, применив то или иное управление (то есть, на значения $W_{i+1}(1)$, $W_{i+1}(2)$, $W_{i+1}(3)$).

Соответственно, уравнение Беллмана для обратной прогонки записывается так:

$$
W_i(s_i) = max_{x_i} \{ w(s_i, x_i) + W_{i+1}(\phi(s_i, x_i)) \}.
$$

Очевидно, любая рекурсия, которую мы хотим вычислить, должна иметь некоторое нерекуррентное основание. Таким основанием для уравнений обратной прогонки выступает *последний этап*:

$$
W_n(s_n) = max_{x_n} \{ w(s_n, x_n) \}.
$$

Для последнего этапа никакого "последующего пути" нет, поэтому оценка условного оптимального выигрыша для всех этапов, начиная с $n$-го ($W_n(s_n)$) определяется выбором наиболее выгодного управления для этапа $n$. Таким образом, вычисление функции Беллмана начинает вычисляться с последнего этапа и постепенно продвигается к первому (отсюда название - *обратная* прогонка).

Метафора для обратной прогонки: функция Беллмана представляет собой наилучший выигрыш, который мы получаем, с учетом "последствий" ("благоразумного поведения" на последующих этапах). Для последнего этапа последствия учитывать уже не надо, поэтому "гуляем на все".

Очень часто возникает следующий вопрос: "Откуда же мы знаем, в каком состоянии окажемся к последнему этапу, какое управление окажется наилучшим?". Действительно, не знаем. И даже не пытаемся предполагать. Идея в том, что $W_i$ и $W_n$ вычисляются для *всех возможных* состояний. Они дословно показывают: "Если к началу соответствующего этапа система находится в определенном состоянии, то это приведет к вот такому оптимальному выигрышу".

## Задача о найме

Смоделируем задачу:

**1. Выделим этапы, выигрыш, управление и состояние.**

Этап: неделя, $i \in \{1, \dots, 5\}$.

Выигрыш: учитываются только денежные потоки. Расходы будем считать со знаком минус (в данной задаче есть только такие потоки).

Управление: количество нанимаемых сотрудников $x_i$.

Состояние: количество сотрудников. Поскольку в уравнение обратной прогонки логичней всего записывается, если берется состояние *к началу* этапа, то под состоянием будем понимать количество сотрудников в начале $i$-того месяца.

Присутствует также ограничение, заключающееся в том, что $s_i + x_i \geq d_i$, где $d_i$ - потребность в сотрудниках на $i$-том этапе.

**2. Уравнение выигрыша на $i$-том этапе (без  учета "последствий")**

Выигрыш определяется всеми экономическими эффектами, связанными с управлением персоналом. А именно:
- наем содрудников;
- содержание избыточной рабочей силы.

$$
w_i(s_i, u_i) = -(400 + 200u_i) \mathbb{I}[u_i > 0] - 300(s_i + u_i - d_i).
$$

Здесь $d_i$ - это потребность в рабочих на $i$-том этапе, $\mathbb{I}[u_i > 0]$ - индикаторная функция, принимающая значение 1 тогда и только тогда, когда выражение $u_i > 0$ истинно. Знак "минус" связан с тем, что все экономические эффекты, рассматриваемые в этой задаче - это расходы.

На Python такая функция может быть записана следующим образом:

In [2]:
# Вычисление выигрыша на заданном этапе.
def profit(stage, s, u):
    return -(400 + 200*u)*int(u > 0) - 300*(s + u - workforce_demand[stage])

**3. Функция перехода для  $i$-того этапа**

Факт найма/увольнения сотрудников изменяет количество рабочих очевидным образом:

$$
\phi(s_i, u_i) = s_i + u_i
$$

In [3]:
def state_transition(s, u):
    return s + u

**4. Записываем уравнение Беллмана для данной задачи**

Объединяем все компоненты задачи и записываем уравнение обратной прогонки:

$$
W_i(s_i) = \max_{u_i + s_i \geq d_i} 
\{ -(400 + 200u_i) \mathbb{I}[u_i > 0]  
   -300(s_i + u_i - d_i) + W_{i+1}(s_i + u_i) 
\}
$$

По условию задачи на начало анализируемого периода рабочих нет. Значит, состояние на начало первого этапа - 0. Следовательно, для получения ответа на вопрос, поставленный в задаче, необходимо вычислить значение функции Беллмана $W_1(0)$.

В силу рекуррентной природы уравнения, начинаем с последнего шага:

In [4]:
# В W_table будем хранить значения функции Беллмана.
# Структура данных следующая:
#  i-тый элемент списка соответствует значениям функции Беллмана для i-того этапа
#  и представляет собой словарь, ключами в котором являются состояния, 
#  а значениями - пары (функция Беллмана, условно-оптимальное управление) 
W_table = [{} for i in range(5)] 

# Заведомо невыгодное значение (при вещественных значениях функции Беллмана
# вместо него можно использовать float('-inf'))
NINF = -1000000

# Начинаем с последнего этапа
# Поскольку мы не знаем, в каком состоянии мы окажемся к началу пятой недели,
# вычисляем функцию Беллмана для всех возможных состояний
# (больше максимального количества сотрудников нанимать, в любом случае, невыгодно):
for s in range(max(workforce_demand)+1):
    W_table[4][s] = (NINF, None)
    # Перебираем только те управления, которые удовлетворяют потребность в персонале
    for hire in range(workforce_demand[4] - s, max(workforce_demand) - s + 1):
        # Убедимся в том, что потребность удовлетворяется:
        assert(s + hire >= workforce_demand[4])
        # Оценим стоимость такого управления
        W = profit(4, s, hire) 
        if W > W_table[4][s][0]:
            W_table[4][s] = (W, hire)

Полученная таблица содержит условно-оптимальный выигрыш и условно-оптимальное управление для всех состояний, возможных к началу последнего этапа. Например, если к началу последней недели будет нанято 0 рабочих, то оптимальным управлением будет:

In [5]:
W_table[4][0]

(-1600, 6)

То есть, если к началу пятой недели нанято 0 рабочих, то необходимо нанять шестерых (это минимально необходимое количество для удовлетворения потребности в рабочей силе). В принципе, расчет для последнего этапа можно было организовать и проще - действительно, на последнем этапе нет смысла нанимать больше рабочих, чем требуется именно для этого этапа, поэтому оптимальное управление всегда будет приводить количество рабочих к требуемому.

Теперь есть все необходимое для вычисления функции Беллмана для предпоследнего этапа:

In [6]:
for s in range(max(workforce_demand)+1):
    W_table[3][s] = (NINF, None)
    # Перебираем только те управления, которые удовлетворяют потребность в персонале
    for hire in range(workforce_demand[3] - s, max(workforce_demand) - s + 1):
        # Убедимся в том, что потребность удовлетворяется:
        assert(s + hire >= workforce_demand[3])
        # Оценим стоимость такого управления (с учетом его "последствий")
        W = profit(3, s, hire) + W_table[4][state_transition(s, hire)][0]
        if W > W_table[3][s][0]:
            W_table[3][s] = (W, hire)

In [7]:
W_table[3]

{0: (-2000, 4),
 1: (-1800, 3),
 2: (-1600, 2),
 3: (-1400, 1),
 4: (-800, 0),
 5: (-800, -1),
 6: (-600, 0),
 7: (-600, -1),
 8: (-600, -2)}

Для остальных этапов принцип подсчета будет точно таким же, поэтому его удобно организовать в виде цикла:

In [8]:
# Да, пересчитаем предпоследний этап тоже:
for i in range(3, -1, -1):
    for s in range(max(workforce_demand)+1):
        W_table[i][s] = (NINF, None)
        # Перебираем только те управления, которые удовлетворяют потребность в персонале
        for hire in range(workforce_demand[i] - s, max(workforce_demand) - s + 1):
            # Убедимся в том, что потребность удовлетворяется:
            assert(s + hire >= workforce_demand[i])
            # Оценим стоимость такого управления (с учетом его "последствий")
            W = profit(i, s, hire) + W_table[i+1][state_transition(s, hire)][0]
            if W > W_table[i][s][0]:
                W_table[i][s] = (W, hire)

Проанализируем таблицу для первой недели:

In [9]:
W_table[0]

{0: (-3300, 5),
 1: (-3100, 4),
 2: (-2900, 3),
 3: (-2700, 2),
 4: (-2500, 1),
 5: (-1900, 0),
 6: (-1900, -1),
 7: (-1800, 0),
 8: (-1800, -1)}

Она вычислена для всех возможных состояний, но нам требуется только одно - соответствующее исходному количеству рабочих 0. По таблице видно, что для этого состояния условно-оптимальным выигрышем будет -3300, а в первую неделю следует нанять 5 рабочих. Управление для второй недели можно определить, если применить функцию перехода с известным начальным состоянием (0) и известным управлением (5):

In [10]:
# К началу второй недели мы окажемся в состоянии:
state_transition(0, 5)

5

In [11]:
# И оптимальным управлением будет:
W_table[1][state_transition(0, 5)]

(-1900, 3)

...нанять еще трех рабочих. Действуя подобным образом можно восстановить управление для всех этапов.

# Прямая прогонка

Уравнения прямой прогонки строятся по другому принципу. Они являются уравнениями для совокупного выигрыша *к концу* $i$-того этапа (который рекуррентно выражается через выигрыш к концу $i-1$-го этапа). Для их построения нужно рассуждать так: если по итогам $i$ шагов система оказывается в состоянии $s_i$, а на $i$-том шаге рассматривается возможность осуществления управления $x_i$, то из какого состояния возможно попасть в $s_i$?  

<img src="images/dp_forward.png" alt="Прямая прогонка" style="width: 600px;"/>

Приципиальным отличием здесь является то, что $W_i(s)$ определяется уже не как *условно-оптимальынй выигрыш начиная с этапа i*, а как *условно-оптимальный выигрыш за первые i этапов* (при условии, что к концу $i$-того этапа система находится в состоянии $s$. Обратите внимание, что уравнения прямой прогонки, в отличие от обратной прогонки, несколько удобней записывать, если считать, что в искомом состоянии система оказывается *к концу* $i$-того этапа.  

Поскольку ДП применяется к задачам, которые удовлетворяют принципу *оптимальности подзадач*, оптимальный путь по пространству состояний задачи от исходного состояния до некоторого $s$ на этапе $i$ включает в себя оптимальный путь по пространству состояний задачи от исходного состояния до $s'$ на этапе $i-1$, из которого применением управления достижимо состояние $s$. Соответственно, мы можем попытаться выразить $W_i(s)$ через $W_{i-1}(s')$ и стоимости управлений, которые приводят из $s'$ в $s$. Так, на приведенной картинке, оценка оптимального выигрыша на всех шагах, до $i$-того включительно, при условии, что система к концу $i$-того этапа находится в состоянии 2 (то есть, $W_i(2)$), опирается на оценку оптимального выигрыша для предыдущих шагов для состояний, из которых возможно попасть в состояние 2, применив то или иное управление (то есть, на значения $W_{i-1}(1)$, $W_{i-1}(2)$, $W_{i-1}(3)$).

Соответственно, уравнение Беллмана для прямой прогонки может быть записано так:

$$
W_i(s_i) = max_{\{x_i | \phi(s_{i-1}, x_i) = s_i\}} \{ w(s_{i-1}, x_i) + W_{i-1}(s_{i-1}) \}.
$$

Основанием для уравнения прямой прогонки выступает обычно некоторый фиктивный этап, предшествующий первому:

$$
W_0(s) = 
    \begin{cases}
      0 & \text{если $s$ - допустимое исходное состояние системы}\\
      -\infty & \text{в противном случае}
    \end{cases} 
$$


Таким образом, вычисление функции Беллмана происходит с первого этапа и постепенно продвигается к последнему (отсюда название - *прямая* прогонка). В любом случае, поскольку изначально неизвестно, какая именно "траектория" приведет к оптимальному решению задачи, вычисление происходит для всех достижимых состояний.

Метафора для прямой прогонки: продвигаясь от первого этапа к последнему, функция Беллмана как бы "накапливает" наилучший способ достигнуть соответствующего состояния к концу $i$-того этапа.

Замечание по реализации:

Уравнение выше предполагает, что вычисляя функцию для некоторого состояния $s$, необходимо определять состояния предыдущего этапа, которые могут привести к $s$. Это соответствует определенному "обращению" функции перехода, что не всегда удобно, поэтому вычисления часто проводят проходя по таблице $i$-того этапа, и модифицируя таблицу следующего этапа.

## Задача о найме

Модель задачи о найме сотрудников примерно такая же, как и для обратной прогонки, однако уравнения выигрыша и перехода записываются несколько иначе (что диктуется несколько другой формой уравнения Беллмана).

Уравнение выигрыша на $i$-том этапе:

$$
w_i(s_{i-1}, u_i) = -(400 + 200u_i) \mathbb{I}[u_i > 0] - 300(s_{i-1} + u_i - d_i).
$$

То есть, если к концу предыдущего ($i-1$-го)этапа было $s_{i-1}$ рабочих, то стоимость управления $w_i(s_{i-1}, u_i)$.

Функция перехода для  $i$-того этапа:

$$
\phi(s_{i-1}, u_i) = s_{i-1} + u_i.
$$

Объединяем все компоненты задачи и записываем уравнение прямой прогонки:

$$
W_i(s_i) = \max_{s_i = s_{i-1} + u_i \\ s_{i-1} + u_i \geq d_i} 
\{ -(400 + 200u_i) \mathbb{I}[u_i > 0] 
   -300(s_{i-1} + u_i - d_i) + W_{i-1}(s_{i-1}) 
\}
$$

In [12]:
# Таблица значений функции Беллмана
W_table = [{} for i in range(6)] 

# Заведомо невыгодное значение (при вещественных значениях функции Беллмана
# вместо него можно использовать float('-inf'))
NINF = -1000000

# Список всех возможных состояний
states = list(range(0, max(workforce_demand)+1))

# Инициализируем значения нулевого этапа (предшествующего первому)
# При этой инициализации следует поставить 0 для тех начальных
# состояний, которые соответствуют условиям задачи.
# В данной задаче перед началом первого этапа (значит, в конце нулевого)
# было 0 рабочих. Следовательно, только это значение должно быть добавлено
# в таблицу нулевого этапа. Для остальных состояний добавляем заведомо
# невыгодные значения:
for s in states:
    W_table[0][s] = (NINF, None)
W_table[0][0] = (0, None)

# Зная значения функции Беллмана для нулевого этапа, можем вычислить 
# для первого.
# Для этого будем перебирать состояния 0-го этапа, возможные управления,
# и корректировать таблицу для 1-го этапа:
for s in states:
    W_table[1][s] = (NINF, None)
for s_prev in states:
    # Перебираем только те управления, которые удовлетворяют потребность в персонале
    for hire in range(workforce_demand[0] - s_prev, max(workforce_demand) - s_prev + 1):
        s = state_transition(s_prev, hire)
        # Убедимся в том, что ограничения соблюдаются:
        assert(s == s_prev + hire)
        assert(s_prev + hire >=  workforce_demand[0])
        # Оценим стоимость такого управления (с учетом условно-оптимального
        # выигрыша получения состояния s_prev к нулевому этапу)
        W = profit(0, s_prev, hire) + W_table[0][s_prev][0]
        if W > W_table[1][s][0]:
            W_table[1][s] = (W, hire)

Полученная таблица содержит оптимальные способы достигнуть каждого допустимого состояния к концу первого этапа (получить определенное число рабочих к концу первого этапа):

In [13]:
W_table[1]

{0: (-1000000, None),
 1: (-1000000, None),
 2: (-1000000, None),
 3: (-1000000, None),
 4: (-1000000, None),
 5: (-1400, 5),
 6: (-1900, 6),
 7: (-2400, 7),
 8: (-2900, 8)}

Для следующих этапов код аналогичен:

In [14]:
for i in range(len(workforce_demand)):
    # Инициализируем таблицу "следующего" этапа:
    for s in states:
        W_table[i+1][s] = (NINF, None)
    for s_prev in states:
        # Перебираем только те управления, которые удовлетворяют потребность в персонале
        for hire in range(workforce_demand[i] - s_prev, max(workforce_demand) - s_prev + 1):
            s = state_transition(s_prev, hire)
            # Убедимся в том, что ограничения соблюдаются:
            assert(s == s_prev + hire)
            assert(s_prev + hire >=  workforce_demand[i])
            # Оценим стоимость такого управления (с учетом условно-оптимального
            # выигрыша получения состояния s_prev к нулевому этапу)
            W = profit(i, s_prev, hire) + W_table[i][s_prev][0]
            if W > W_table[i+1][s][0]:
                W_table[i+1][s] = (W, hire)

К последнему шагу:

In [15]:
W_table[5]

{0: (-1000000, None),
 1: (-1000000, None),
 2: (-1000000, None),
 3: (-1000000, None),
 4: (-1000000, None),
 5: (-1000000, None),
 6: (-3300, 0),
 7: (-3900, 0),
 8: (-4500, 4)}

Максимальная стоимость будет в том случае, если к концу последнего этапа окажется нанятыми 6 рабочих.

Для восстановления управления следует двигаться в обратном направлении. Так, мы знаем, что оптимальным состоянием к концу последнего этапа является 6, и при этом условно-оптимальным управлением является управление 0 (не нанимать рабочих). Значит, к концу предпоследнего этапа должно было быть $6 - 0 = 6$ рабочих:

In [16]:
W_table[4][6]

(-3300, -2)

На предпоследнем этапе предполагается уволить двух рабочих и т.д.

In [17]:
def restore_forward(t, stage, state):
    control = []
    while stage != 0:
        control.insert(0, t[stage][state][1])
        state = state - t[stage][state][1]
        stage -= 1
    return control

In [18]:
restore_forward(W_table, 5, 6)

[5, 3, 0, -2, 0]